<a href="https://colab.research.google.com/github/kmkarakaya/ML_tutorials/blob/master/webScrapping_FindYourDreamCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title
cd drive/My Drive/Colab Notebooks

In [ ]:
#@title Import Libraries

from __future__ import print_function
import requests # for making standard html requests
from bs4 import BeautifulSoup # magical tool for parsing html data
import json # for parsing data
import pandas as pd # premier library for data organization
import numpy as np
from joblib import dump, load  
from sklearn.compose import ColumnTransformer,  make_column_transformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import NearestNeighbors 

In [90]:
#@title Web Scrapping functions
def getCarLinks(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  carSet= set()
  carLinks = soup.find_all(class_ = 'table listing-table w100 border-grey2')
  for i in carLinks:
    #print(i)
    carLink = i.find_all(class_ ='crop-after')
    #print("carLink: ", carLink)
    for j in carLink:
      #print(j.find('a')['href'])
      carSet.add(baseUrl+j.find('a')['href'])
  return carSet

def getCarList(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  carList= list()
  #page.encoding = 'ISO-885901'
  #soup = BeautifulSoup(page.text, 'html.parser')
  #ul class="w100 cf mt16"
  arabaOzellikleri = soup.find_all(class_ = 'df df-center w100')
  
  for i in arabaOzellikleri:
    print("arabaOzellikleri:\n", i)
    fiyat= i.find('span')
    print("Price:\n", fiyat)  
    print(fiyat.text.strip())
    arabaDic['fiyat'] = fiyat.text.strip()
  print(arabaDic)
  araba = soup.find_all(class_ = 'w100 cf mt16')
  print(araba)
  arabaOzellikleri= araba[0].find_all(['span','a'])

  for idx, i  in enumerate(arabaOzellikleri):
    #print("arabaOzellikleri:\n", i)
    if (idx%2==0):
          key = i.text.strip()
    else:
          val = i.text.strip()
          arabaDic[key]=val
      
    print(idx, " : ", i.text.strip())
  print(arabaDic)
  
  return arabaDic
  


def getCarInfo(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  arabaDic= dict()
  #page.encoding = 'ISO-885901'
  #soup = BeautifulSoup(page.text, 'html.parser')
  #ul class="w100 cf mt16"
  arabaOzellikleri = soup.find_all(class_ = 'df df-center w100')
  if (len(arabaOzellikleri)==0): #kredi reklamı
    arabaOzellikleri = soup.find_all(class_ = 'df df-center df-wrap w66 fl')
  for i in arabaOzellikleri:
    #print("arabaOzellikleri:\n", i)
    fiyat= i.find('span')
    #print("fiyat:\n", fiyat)  
    #print(fiyat.text.strip())
    arabaDic['fiyat'] = fiyat.text.strip()
  #print(arabaDic)
  araba = soup.find_all(class_ = 'w100 cf mt16')
  #print(araba)
  arabaOzellikleri= araba[0].find_all(['span','a'])

  for idx, i  in enumerate(arabaOzellikleri):
    #print("arabaOzellikleri:\n", i)
    if (idx%2==0):
          key = i.text.strip()
    else:
          val = i.text.strip()
          arabaDic[key]=val
      
    #print(idx, " : ", i.text.strip())
  print(arabaDic)
  
  return arabaDic


def getCars(urls, allCars):
  
  for url in urls:
    carDic=getCarInfo(url)

    #if (set(resultColumns)!=set(carDic.keys())):
    allCars= allCars.append(carDic, ignore_index=True)
    
  print(len(allCars)," cars downloaded!\n\n")
  return allCars
  
def getPageLinks(nextPage):
  pageSet= set()
  print("initial page", nextPage)
  while (nextPage is not None):
    pageSet.add(nextPage)
    page = requests.get(nextPage)
    soup = BeautifulSoup(page.text, 'html.parser')
    pageLinks = soup.find(class_ = 'pagination')
    nextPage=None
    #print("pageLinks: \n", pageLinks)
    for i in pageLinks.find_all(class_ = 'arrowed'):
      if (i.find('a')!= -1) and (i.find('a')is not None):
        page=i.find('a')['href']
        page= baseUrl+page
        if page not in pageSet:
          nextPage = page
          print("find next page", nextPage)
  print( "Found pages:\n")
  #display(pageSet)
  return list(pageSet)

In [ ]:
resultColumns = ['fiyat', 'İlan No:', 'İlan Tarihi:', 'Marka:', 'Seri:', 'Model:',
       'Yıl:', 'Yakıt Tipi:', 'Vites Tipi:', 'Motor Hacmi:', 'Motor Gücü:',
       'Kilometre:', 'Boya-değişen:', 'Takasa Uygun:', 'Kimden:']
allCars = pd.DataFrame(columns=resultColumns)

carDic= dict()

#@markdown Specify Your Car Type for webscraping
carType = "320d" #@param {type:"string"}
baseUrl='https://www.arabam.com/'
url = "https://www.arabam.com/ikinci-el?searchText="+carType+"&take=50"
searchResult = list()
pageLinks=getPageLinks(url)
print("Total pages: ", len(pageLinks))
pageNo=1
for pageURL in pageLinks:
  print("Loading Page: {}".format(pageNo))
  pageNo= pageNo +1
  searchResult= list(getCarLinks(pageURL))
  print("search result: {}".format(len(searchResult))) 
  allCars=getCars(searchResult,allCars)
searchTerms = carType.replace(" ","_")
fileName = searchTerms+".csv"

selectedFeatures = ["fiyat", "Yıl:", "Kilometre:", "Boya-değişen:", "Yakıt Tipi:", "Vites Tipi:", "Model:"]
allCars=allCars.dropna(subset=selectedFeatures, how = 'any')
allCars = allCars[allCars['Kimden:'] != 'Rent a Car']
numeric_inputs = ['fiyat', 'Kilometre:']
for feature in numeric_inputs:
    allCars[feature] = allCars[feature].str.replace(r'\D', '').astype(int)

allCars.to_csv(fileName,index=False)
print(" Downloaded cars stored to "+fileName)

In [82]:
#@title NearestNeighbors Unsupervised Learning
def clearOutput(b):
  from IPython.display import clear_output
  clear_output(wait=False)
  print("clear!")

def predict(b):
  allCars = pd.read_csv(fileName)
  dreamCarIndex=len(allCars) 
  pd.set_option('display.max_columns', None)

  d = {'fiyat': [price.value],'Kilometre:': [km.value], 'Yıl:': [year.value], 
      'Boya-değişen:': [body.value], "Vites Tipi:": [gear.value],
      "Yakıt Tipi:": gas.value, "Model:": model.value}
  #print("Query: ", d)
  dreamCar= pd.DataFrame(data=d)
  
  support = allCars.apply(lambda x: True if x['Yıl:']== year.value and  
                          x['Model:']== model.value and x['Yakıt Tipi:']== 
                          gas.value and x['Vites Tipi:']== gear.value  
                          else False , axis=1)
  support  = len(support[support == True].index)
  support = support /allCars.shape[0]
  
  allCars = allCars[selectedFeatures]
  allCars= allCars.append(dreamCar, verify_integrity =True, ignore_index=True)
  print('\n\n************* Your Dream Car is : *************\n')
  display(allCars.tail(1))
  print ("support for your query {:.2f}%".format(100*support)) 

  numeric_features = ['fiyat', 'Kilometre:']
  categorical_features = ["Yıl:","Boya-değişen:", "Yakıt Tipi:"\
                          , "Vites Tipi:", "Model:"]
  X = allCars[selectedFeatures]
  preprocess = make_column_transformer(
     ( MinMaxScaler(), numeric_features),
     ( OneHotEncoder(), categorical_features)
     )
  X=pd.DataFrame(preprocess.fit_transform(X).toarray() )
  neigh = NearestNeighbors(algorithm='kd_tree')
  neigh.fit(X)
  result=neigh.kneighbors([X.tail(1)], 5, return_distance=True)[1][0]
  #print(result)
  result = np.setdiff1d(result,np.array(dreamCarIndex))
  #print(result)
  print('\n         Similar cars to your dream:\n')
  display(allCars.iloc[result,:])
  

  return result

In [94]:
  #@title Enter Your Dream Car :)
  from ipywidgets import interact, interactive, fixed, interact_manual
  import ipywidgets as widgets

  
  allCars = pd.read_csv(fileName) 
  
  #modelName = searchTerms+'.joblib'
  #clf= load(modelName)
  
  #allCars, clf= createTrain(allCars, True)
  
  price= widgets.IntSlider(description='Price',min=allCars['fiyat'].min(), max=allCars['fiyat'].max(), step=1, value=allCars['fiyat'].min())
  year=widgets.IntSlider(description='Year',min=allCars['Yıl:'].min(), max=allCars['Yıl:'].max(), step=1, value=allCars['Yıl:'].min())
  km=widgets.IntSlider(description='km',min=allCars['Kilometre:'].min(), max=allCars['Kilometre:'].max(), step=1, value=allCars['Kilometre:'].min())
  body=widgets.Combobox(description='Body',options=list(set(allCars['Boya-değişen:'])))
  gas=widgets.Combobox(description='Fuel',options=list(set(allCars['Yakıt Tipi:'])))
  gear=widgets.Combobox(description='Gear',options=list(set(allCars['Vites Tipi:'])))
  model=widgets.Combobox(description='Model',options=list(set(allCars['Model:'])))

  btn = widgets.Button(description="Find My Dream Car!")
  btn.on_click(predict)
  btnClear = widgets.Button(description="Clear")
  btnClear.on_click(clearOutput)

  ui = widgets.VBox([price, year, km, body,gas, gear,model,btn,btnClear])
  def f(price, year, km, body,gas, gear,model):
      #print((price, year, km, body,gas, gear,model))
      print()
  out = widgets.interactive_output(f, {'price':price,'year': year, 'km': km, 'body': body, 'gas': gas, 'gear' : gear, 'model':model })

  display(ui, out)



Output()



************* Your Dream Car is : *************



,fiyat,Yıl:,Kilometre:,Boya-değişen:,Yakıt Tipi:,Vites Tipi:,Model:
549,141776,2004,189014,1 boyalı,Dizel,Düz,320d xDrive Comfort Plus


support for your query 0.00%

         Similar cars to your dream:



,fiyat,Yıl:,Kilometre:,Boya-değişen:,Yakıt Tipi:,Vites Tipi:,Model:
61,135000,2011,156000,"1 değişen, 2 boyalı",Dizel,Otomatik,320d Standart
94,150000,2012,158000,"2 değişen, 2 boyalı",Dizel,Otomatik,320d Luxury
292,148000,2010,136000,Tamamı orjinal,Dizel,Otomatik,320d Standart
349,137500,2011,153000,"2 değişen, 1 boyalı",Dizel,Otomatik,320d Comfort
394,134750,2010,159000,10 boyalı,Dizel,Otomatik,320d Standart
